<a href="https://colab.research.google.com/github/nickleus27/CST383_SportsData/blob/main/SportsDataNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BitByBit Team: Sports Data Notebook
<ul>
<li>Nick Anderson</li>
<li>Oscar Ochoa</li>
<li>Juan Duarte</li>
</ul>

<hr>

# Add Installs:

# Add Imports:



In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


# Read Data:

In [2]:
# df = pd.read_csv()